# Intro to SQLAlchemy and Gevent
The following code was written to use the pgbench database. In this case it was created with the following command: `pgbench -i -s 400 pgbench`. The timings in this file were from a Macbook Pro 2017 with an i5 running at 2.5Ghz and 8GB of RAM.

In [1]:
from sqlalchemy import MetaData, create_engine
from sqlalchemy.ext.automap import automap_base
metadata = MetaData()
Base = automap_base()
engine = create_engine('postgresql+psycopg2://jasonmyers@localhost:5432/pgbench')
Base.prepare(engine, reflect=True)

In [2]:
Accounts = Base.classes.pgbench_accounts

In [3]:
from sqlalchemy.orm import Session
session = Session(engine)

In [4]:
from sqlalchemy import func
query = session.query(Accounts.bid, func.count(1)).group_by(Accounts.bid).limit(5000)

In [5]:
def execute_query():
    query.all()

In [6]:
%%time
print(execute_query())

None
CPU times: user 3.08 ms, sys: 1.54 ms, total: 4.62 ms
Wall time: 24.9 s


In [7]:
import threading
def async_creation_runner(cache, somekey, creator, mutex):
    def runner():
        try:
            value = creator()
            cache.set(somekey, value)
        finally:
            mutex.release()

    thread = threading.Thread(target=runner)
    thread.start()

In [8]:
from dogpile.cache.util import sha1_mangle_key
def unicode_sha1_mangle_key(key):
    return sha1_mangle_key(key.encode('ascii', 'ignore'))


def mangle_key(key):
    prefix, key = key.split(':', 1)
    base = 'cookie:cache:'
    if prefix:
        base += '{}'.format(prefix)
    else:
        raise ValueError(key)
    value = '{}:{}'.format(base, unicode_sha1_mangle_key(key))
    return value

In [9]:
import sys
from dogpile.cache import make_region
regions = {}

regions['default'] = make_region(async_creation_runner=async_creation_runner,
                                 key_mangler=mangle_key).configure(
    'dogpile.cache.redis',
    arguments={
        'host': 'localhost',
        'port': 6379,
        'db': 0,
        'redis_expiration_time': 60*60*2,   # 2 hours
        'distributed_lock': True,
        'lock_timeout': 120,
        'lock_sleep': 5
    }
)


In [10]:
def _key_from_query(query, qualifier=None):
    stmt = query.with_labels().statement
    compiled = stmt.compile()
    params = compiled.params

    return " ".join([str(compiled)] +
                    [str(params[k]) for k in sorted(params)])

In [11]:
from sqlalchemy.orm.query import Query
from dogpile.cache.api import NO_VALUE


class CachingQuery(Query):

    def __init__(self, regions, *args, **kw):
        self.cache_regions = regions
        self.saved_to_cache = False
        Query.__init__(self, *args, **kw)

    def __iter__(self):
        if hasattr(self, '_cache_region'):
            return self.get_value(
                createfunc=lambda: list(Query.__iter__(self)))
        else:
            return Query.__iter__(self)
        
    def _get_cache_plus_key(self):
        dogpile_region = self.cache_regions[self._cache_region.region]
        if self._cache_region.cache_key:
            key = self._cache_region.cache_key
        else:
            key = _key_from_query(self)
        return dogpile_region, key
    
    def get_value(self, merge=True, createfunc=None,
                  expiration_time=None, ignore_expiration=False):
        dogpile_region, cache_key = self._get_cache_plus_key()

        assert not ignore_expiration or not createfunc, \
            "Can't ignore expiration and also provide createfunc"

        if ignore_expiration or not createfunc:
            cached_value = dogpile_region.get(
                cache_key,
                expiration_time=expiration_time,
                ignore_expiration=ignore_expiration
            )
        else:
            try:
                cached_value = dogpile_region.get_or_create(
                    cache_key,
                    createfunc,
                    expiration_time=expiration_time
                )
            except ConnectionError:
                logger.error('Cannot connect to query caching backend!')
                cached_value = createfunc()
        if cached_value is NO_VALUE:
            raise KeyError(cache_key)
        if merge:
            cached_value = self.merge_result(cached_value, load=False)
        return cached_value
    
    def set_value(self, value):
        dogpile_region, cache_key = self._get_cache_plus_key()
        try:
            dogpile_region.set(cache_key, value)
            self.saved_to_cache = True
        except ConnectionError:
            logger.error('Cannot connect to query caching backend!')

In [12]:
from sqlalchemy.orm.interfaces import MapperOption

class FromCache(MapperOption):
    """Specifies that a Query should load results from a cache."""

    propagate_to_loaders = False

    def __init__(self, region="default", cache_key=None, cache_prefix=None):
        self.region = region
        self.cache_key = cache_key
        self.cache_prefix = cache_prefix

    def process_query(self, query):
        query._cache_region = self

In [13]:
def query_callable(regions, query_cls=CachingQuery):
    def query(*arg, **kw):
        return query_cls(regions, *arg, **kw)
    return query

In [14]:
from sqlalchemy.orm import sessionmaker

def init_caching_session(engine=None):
    if not engine:
        return

    return sessionmaker(
        bind=engine, autoflush=False, autocommit=False,
        query_cls=query_callable(regions)
    )

In [15]:
CachingSession = init_caching_session(engine)
caching_session=CachingSession()

In [16]:
query = caching_session.query(Accounts.bid, func.count(1)).group_by(Accounts.bid).limit(5000).options(
            FromCache('default'))

In [17]:
%%time
print(execute_query())

None
CPU times: user 10.4 ms, sys: 7.3 ms, total: 17.7 ms
Wall time: 24.8 s


In [18]:
%%time
print(execute_query())

None
CPU times: user 3.5 ms, sys: 1.62 ms, total: 5.11 ms
Wall time: 4.32 ms
